In [1]:
import kashgari
kashgari.config.use_cudnn_cell = True

╭─────────────────────────────────────────────────────────────────────────╮
│ ◎ ○ ○ ░░░░░░░░░░░░░░░░░░░░░  Important Message  ░░░░░░░░░░░░░░░░░░░░░░░░│
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│              We renamed again for consistency and clarity.              │
│                   From now on, it is all `kashgari`.                    │
│  Changelog: https://github.com/BrikerMan/Kashgari/releases/tag/v1.0.0   │
│                                                                         │
│         | Backend          | pypi version   | desc           |          │
│         | ---------------- | -------------- | -------------- |          │
│         | TensorFlow 2.x   | kashgari 2.x.x | coming soon    |          │
│         | TensorFlow 1.14+ | kashgari 1.x.x |                |          │
│         | Keras            | kashgari 0.x.x | legacy version |          │
│           

In [2]:
from kashgari.corpus import ChineseDailyNerCorpus

train_x, train_y = ChineseDailyNerCorpus.load_data('train')
valid_x, valid_y = ChineseDailyNerCorpus.load_data('valid')
test_x, test_y = ChineseDailyNerCorpus.load_data('test')

In [3]:
# from kashgari.corpus import CONLL2003ENCorpus

# train_x, train_y = CONLL2003ENCorpus.load_data('train')
# valid_x, valid_y = CONLL2003ENCorpus.load_data('valid')
# test_x, test_y = CONLL2003ENCorpus.load_data('test')

# train_x = [['Hello', 'world'], ['Hello', 'Kashgari'], ['I', 'love', 'Beijing']]
# train_y = [['O', 'O'], ['O', 'B-PER'], ['O', 'B-LOC']]

# valid_x, valid_y = train_x, train_y
# test_x, test_x = train_x, train_y

In [11]:
print(train_x[2])
print(len(train_x[0]))
print(len(train_x))

['有', '的', '学', '者', '指', '出', '，', '不', '应', '当', '把', '改', '良', '与', '革', '命', '对', '立', '起', '来', '，', '改', '良', '是', '社', '会', '革', '命', '的', '准', '备', '和', '前', '奏', '，', '革', '命', '是', '解', '决', '改', '良', '所', '无', '法', '解', '决', '的', '那', '些', '根', '本', '性', '的', '全', '局', '性', '的', '社', '会', '问', '题', '，', '中', '国', '近', '代', '的', '进', '步', '政', '治', '运', '动', '，', '不', '管', '是', '温', '和', '的', '改', '革', '，', '或', '是', '暴', '力', '革', '命', '，', '都', '是', '为', '了', '实', '现', '近', '代', '化', '这', '个', '目', '的', '。']
20
20864


In [5]:
print(train_y[2])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [8]:
import kashgari
from kashgari.tasks.labeling.models import BiLSTM_Model

model = BiLSTM_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10)
model.evaluate(test_x, test_y)
model.save('model/BiLSTM')
loaded_model = kashgari.utils.load_model('model/BiLSTM')
# loaded_model.predict(test_x[:10])

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 97)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 97, 100)           350000    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 97, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 97, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 97, 8)             2056      
_________________________________________________________________
activation_2 (Activation)    (None, 97, 8)             0         
Total params: 587,576
Trainable params: 587,576
Non-trainable params: 0
_____________________________________________________

UnknownError: 2 root error(s) found.
  (0) Unknown: Fail to find the dnn implementation.
	 [[{{node layer_blstm_2/CudnnRNN_1}}]]
	 [[loss_2/mul/_399]]
  (1) Unknown: Fail to find the dnn implementation.
	 [[{{node layer_blstm_2/CudnnRNN_1}}]]
0 successful operations.
0 derived errors ignored.

In [9]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_Model
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=100)

model = BiLSTM_Model(bert_embed)
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10)
model.evaluate(test_x, test_y)
model.save('model/BERT_BiLSTM')
loaded_model = kashgari.utils.load_model('model/BERT_BiLSTM')

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 768)     1536        Input-Segment[0][0]              
___________________________________________________________________________________________

Epoch 1/10


UnknownError: 2 root error(s) found.
  (0) Unknown: Fail to find the dnn implementation.
	 [[{{node layer_blstm_3/CudnnRNN_1}}]]
	 [[loss_3/mul/_2123]]
  (1) Unknown: Fail to find the dnn implementation.
	 [[{{node layer_blstm_3/CudnnRNN_1}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_CRF_Model
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING,
                           sequence_length=100)

model = BiLSTM_CRF_Model(bert_embed)
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10)
model.evaluate(test_x, test_y)
model.save('model/BERT_BiLSTM_CRF')
loaded_model = kashgari.utils.load_model('model/BERT_BiLSTM_CRF')